<a href="https://colab.research.google.com/github/niranjan-7/ai-chatbot/blob/main/ai_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tflearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 107 kB 25.2 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=1da1707bf2b31e317afa51d28ec0b4034e5270d530847650a3b29281ea45604a
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


In [ ]:
import nltk
nltk.download("punkt")
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()


import tensorflow as tf  
import numpy as np
import tflearn       
import random
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
with open("/content/sample_data/intents.json") as json_data: 
    intents = json.load(json_data)

In [ ]:
print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hello, thanks for visiting', 'Good to see you again', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later, thanks for visiting', 'Have a nice day', 'Bye! Come back again soon.']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'hours', 'patterns': ['What hours are you open?', 'What are your hours?', 'When are you open?', 'When is the time to contact ?', 'At what time do you provide services ?'], 'responses': ["We're open every day from 9AM to 9PM", 'Our working hours are 9AM to 9PM every day']}, {'tag': 'location', 'patterns': ['What is your location?', 'Where are you located?', 'What is your address?', 'Where is your company situated?', 'How can we contact you ?', 'How can I 

In [ ]:
words=[]
documents = []
classes = []


# This list will be used for ignoring all unwanted punctuation marks.
ignore = ["?"]

# Starting a loop through each intent in intents["patterns"]
for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        
        # tokenizing each and every word in the sentence by using word tokenizer and storing in w
        w = nltk.word_tokenize(pattern) 
        #print(w)
        
        # Adding tokenized words to words empty list that we created
        words.extend(w) 
        #print(words)
        
        # Adding words to documents with tag given in intents file
        documents.append((w, intent["tag"]))
        #print(documents)
        
        # Adding only tag to our classes list
        if intent["tag"] not in classes:      
            classes.append(intent["tag"])  #If tag is not present in classes[] then it will append into it.
            print(classes)

['greeting']
['greeting', 'goodbye']
['greeting', 'goodbye', 'thanks']
['greeting', 'goodbye', 'thanks', 'hours']
['greeting', 'goodbye', 'thanks', 'hours', 'location']
['greeting', 'goodbye', 'thanks', 'hours', 'location', 'payments']
['greeting', 'goodbye', 'thanks', 'hours', 'location', 'payments', 'services']
['greeting', 'goodbye', 'thanks', 'hours', 'location', 'payments', 'services', 'training']
['greeting', 'goodbye', 'thanks', 'hours', 'location', 'payments', 'services', 'training', 'Cryptography']
['greeting', 'goodbye', 'thanks', 'hours', 'location', 'payments', 'services', 'training', 'Cryptography', 'Firewall']
['greeting', 'goodbye', 'thanks', 'hours', 'location', 'payments', 'services', 'training', 'Cryptography', 'Firewall', 'Penetration Testing']
['greeting', 'goodbye', 'thanks', 'hours', 'location', 'payments', 'services', 'training', 'Cryptography', 'Firewall', 'Penetration Testing', 'Vulnerability Assessment']
['greeting', 'goodbye', 'thanks', 'hours', 'location', '

In [ ]:
#Performing Stemming by using stemmer.stem() nd lower each word 
#Running loop in words[] and ignoring punctuation marks present in ignore[]

words = [stemmer.stem(w.lower()) for w in words if w not in ignore]  
words = sorted(list(set(words)))  #Removing Duplicates in words[]

#Removing Duplicate Classes
classes = sorted(list(set(classes)))

#Printing length of lists we formed
print(len(documents),"Documents \n")
print(len(classes),"Classes \n")
print(len(words), "Stemmed Words ")

106 Documents 

37 Classes 

118 Stemmed Words 


In [ ]:
training = []
output = []

#Creating empty array for output
output_empty = [0] * len(classes)

for doc in documents:
    bag = [] 
    pattern_words = doc[0]
    #print(pattern_words)
    
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]  
    
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] =1 
    training.append([bag, output_row])

In [ ]:
random.shuffle(training) 
training = np.array(training) 

train_x = list(training[:,0])
train_y = list(training[:,1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [ ]:
# tf.reset_default_graph() #Reset Underlying Graph data
tf.compat.v1.reset_default_graph()
#Building our own Neural Network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation="softmax")
net = tflearn.regression(net)


model = tflearn.DNN(net, tensorboard_dir="tflearn_logs") 

model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True) 
model.save("model.tflearn") #Saving the model

Training Step: 13999  | total loss: 0.00548 | time: 0.096s
| Adam | epoch: 1000 | loss: 0.00548 - acc: 1.0000 -- iter: 104/106
Training Step: 14000  | total loss: 0.00526 | time: 0.123s
| Adam | epoch: 1000 | loss: 0.00526 - acc: 1.0000 -- iter: 106/106
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [ ]:

import pickle

pickle.dump({"words":words, "classes":classes, "train_x":train_x, "train_y":train_y}, open("training_data", "wb"))

In [ ]:
data = pickle.load(open("training_data","rb"))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [ ]:
with open("/content/sample_data/intents.json") as json_data: 
    intents = json.load(json_data)

In [ ]:
model.load("./model.tflearn")

INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [ ]:
def clean_up_sentence(sentence):
    
    sentence_words = nltk.word_tokenize(sentence)
    
    sentence_words= [stemmer.stem(word.lower()) for word in sentence_words]

    return sentence_words

def bow(sentence, words, show_details=False):
    
    sentence_words = clean_up_sentence(sentence)
    
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("Found in bag: %s"% w)
    return(np.array(bag))

In [ ]:
context = {} 

ERROR_THRESHOLD = 0.25
def classify(sentence):
    
    results = model.predict([bow(sentence, words)])[0]
    
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    if results:
        while results:
            for i in intents['intents']:
                
                if i['tag'] == results[0][0]:
                    
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        
                        return print(random.choice(i['responses']))

            results.pop(0)

In [ ]:
response("what kind of location do you stay in?")

Any time!
